<h1 style="text-align:center;">Informe Estructura de Datos</h1>

---

Grupo 4.

Integrantes:

-Massimino, Agustín

-Miguel, Emmanuel

-Tubio, Kevin

-Caceres Montenegro, Nicolas Benjamin


*INTRODUCCIÓN*

Por medio del presente informe se busca explicar ciertas decisiones de diseño y resoluciones acerca del trabajo que consistía en la obtención de tweets y su previa manipulación, generando un buscador el cual permita encontrar tweets segun el usuario, la fecha y hora, palabras o frases del tweet. 
Se solicitó que la implementación utilice indices invertidos y se genere una interfaz la cual permita la utilización del buscador

_DIAGRAMA DE CLASES_

[https://drive.google.com/file/d/1q1NZr0pBG_p8Oog9_iIxb2jVwRVo1si7/view?usp=sharing](https://drive.google.com/file/d/1q1NZr0pBG_p8Oog9_iIxb2jVwRVo1si7/view?usp=sharing)

_DESARROLLO_

El primer desafío que nos encontramos fue una interfaz la cual sea amigable con el usuario pero a su vez que cumpla con lo solicitado. Para lograrlo, generamos una clase llamada "UI" ubicada en "interfaz_usuario.py" en la cual utilizamos lo que llamamos "Menú plegable" el cual muestra una serie de opciones según en la etapa que se esté.
Para la etapa inicial se ejecuta el siguiente menú

In [1]:
def __desplegar_menu(self):
        print("|************************************************************************|")
        print("| 1) Descargar Tweets                                                    |")
        print("| 2) Indexar (Necesario para buscar)                                     |")
        print("| 3) Buscador de Tweets                                                  |")
        print("| 4) Cerrar                                                              |")
        print("|________________________________________________________________________|")
        print()

Como se observa, el mismo permite ingresar a las distintas funcionalidades de nuestro programa y se encuentran ordenadas en como deberían ser ejecutadas. Primero se debería descargar tweets, luego indexar y finalmente buscar. El programa puede cerrarse utilizando "CTRL+C" o utilizando la opción correspondiente.

Para la selección de opciones, decidimos utilizar un diccionario, ya que este permite vincular un número (key) con un valor (llamado de función) y así evitamos tener que crear varias estructuras condicionales para su control o un switch. Esto se logra de la siguiente manera:

In [ ]:
def accion(self):
        self._ejecutando = True
        while self._ejecutando:
            try:
                self.__desplegar_menu()
                n = input("")
                self.limpiar_consola()
                opciones = {
                    "1": self._descargador.descargar,
                    "2": self.indexar,
                    "3": self.__buscador_menu,
                    "4": self.cerrar_programa,
                }
                opciones.get(n, lambda: self.desplegar_mensaje("Opcion invalida."))()
            except OperacionCanceladaException as e:
                print(e)
            except KeyboardInterrupt:
                self.cerrar_programa()

Podemos ver que el método se fija que el atributo ejecutando sea True (siempre será True salvo cuando se ejecuta el método cerrar_programa). Luego llama al menú y espera un input del usuario. Luego intenta obtener el valor del input en el diccionario y si este no existe, arrojará un mensaje formateado (para eso es el método desplegar_mensaje) diciendo que la opción es inválida y se volverá a desplegar el menú. En caso de que la opción exista, pone el valor pero le agrega parentesis, ya que el llamado de las funciones los necesita para ejecutarse.

Como veran, la primer opción llama a un atributo que hace referencia a un objeto. Para que se entienda mejor, indicamos a continuación todos los atributos que posee la clase del interfaz:

In [2]:
def __init__(self):
        self._buscador = Buscador()
        self._indexador = Indexador()
        self._descargador = TweetDownloader()

Cada uno de estos atributos es un objeto perteneciente a otra clase. El atributo _buscador_ pertenece a la clase "Buscador" que se encuentra en "buscador.py"; el atributo _indexador_ pertenece a la clase "indexador" y se encuentra en "indexador.py" y por último, el atributo _descargador_ pertenece a la clase TweetDownloader ubicada en "tweet_downloader.py"

1) DESCARGAR TWEETS



La opción de descarga de tweets llama directamente a un método del atributo _descargador_. En esta clase se utiliza la TwitterAPI por lo que fue obligatorio el uso de ciertos métodos para el agregado de reglas, eliminado de reglas, comenzar stream, entre otras.
En este paso, nos encontramos con un gran problema: La velocidad de descarga.

Nuestra query (llamada así a las reglas que se aplican para la descarga de tweets) inicial no lograba obtener una cantidad significativa de tweets. Estaba descargando aproximadamente unos 100 tweets cada 15 minutos, algo que nos complicaría la obtención de una masa útil de tweets para el proyecto.
Tras experimentar con la query, nos dimos cuenta que ampliando el rango de idiomas (al comienzo, solo buscabamos en español), el resultado podría ser totalmente distinto, además decidimos ampliar los resultandos insertando otras monedas (ya que inicialmente buscabamos solo por bitcoin).
El cambio fue abismal, pasamos a descargar 1000 tweets por minuto.

La query en cuestión quedo de la siguiente forma:

In [ ]:
query = 'bitcoin OR crypto OR "blockchain"'
query2 = '"ethereum" OR "dogecoin" OR "cardano"'
query3 = '"DOGE" OR "XRP"'
query4 = 'BTC OR ETH'
filters = '(lang:es OR lang:en) -is:reply -is:retweet -has:videos -has:links'
self.__agregar_regla_al_stream(f'{query} {filters}')
self.__agregar_regla_al_stream(f'{query2} {filters}')
self.__agregar_regla_al_stream(f'{query3} {filters}')
self.__agregar_regla_al_stream(f'{query4} {filters}')

Ahora, la descarga de tweets ya no era un problema, pero teníamos que asegurarnos que siempre se guarden los tweets, ya que sino se estaría perdiendo información.
Para no tener que grabar en el CSV de forma constante, se estableció que cada 1.000 tweets descargados, se guarde en disco. Además, toda excepción que signifique una finalización de la ejecución (por ejemplo, un KeyboardInterrupt, una perdida de conexión con twitter, entre otros) llama al método que guarda en disco, para que de esta forma, pase lo que pase, se puedan guardar en disco los tweets ya descargados hasta el momento.

_¿Por qué CSV?_

En conjunto, decidimos que un CSV sería una opción adecuada para el guardado de tweets, ya que estos se guardarían con headers lo que nos permitiría facilmente acceder a cada dato y manipularlo a nuestra conveniencia. Por ejemplo, creamos un método el cual guarda unicamente que los datos que utilizaremos además de brindarles formato a los mismos:

In [ ]:
 def __quitar_atributos_innecesarios(self, tweet : dict) -> dict:
        aux = {}
        fecha = datetime.fromisoformat(tweet['data']['created_at'][:-1])
        aux['fecha'] = fecha.strftime("%d/%m/%Y")
        aux['hora'] = fecha.strftime("%H:%M")
        aux['id'] = tweet['data']['id']
        aux['author_id'] = tweet['data']['author_id']
        aux['username'] = tweet['data']['author_id_hydrate']['username']
        aux['text'] = tweet['data']['text']
        return aux

Finalmente, nos encontramos con un último desafío, indicarle al usuario que el proceso de descarga estaba funcionando de forma adecuada.
Para esto, decidimos generar un nuevo hilo el cual se encargue exclusivamente de generar una animación en pantalla y de actualizar constantemente la cantidad de tweets descargados.
Resumidamente, este hilo itera entre una lista de strings, de forma tal que se visualiza como una barra que se desliza de izquierda a derecha. De esta forma, el usuario siempre podrá ver si la ejecución sigue en curso.

2) INDEXAR

El indexador fue en lo que invertimos la mayor cantidad de nuestro tiempo. Debíamos lograr hacer un método que sea capaz de controlar miles de tweets a la vez y generar distintos indices invertidos.
En nuestro caso, optamos por 4 indices invertidos: Usuarios, Fechas y horas, Palabras y Texto (siendo texto lo tweeteado).

Para comenzar, en la inicialización del indexador se definen la cantidad de tweets por bloque (por defecto son 10.000) y otros atributos propios de la clase como las stopwords (tanto en español como en inglés), los distintos diccionarios y una validación de directorios necesaria para que en caso de no existir las carpetas donde se guardaran los archivos, estas sean creadas.

Optamos por utilizar un indice BSBI (Blocked sort-based indexing) ya que, de esta forma podríamos generar distintos bloques (como dijimos, cada 10.000 tweets) los cuales serían archivos temporales/intermedios que luego se ordenarían y unirían en un solo archivo final.
Por un lado se generan los diccionarios, los cuales contienen clave : id (por ejemplo, en el caso de los usuarios sería Usuario : id), y por otro lado se guarda el posting del mismo (para así lograr formar el indice invertido) el cual tendrá de forma ordenada, listas de tweetsID que corresponden a esa clave.

Siguiendo con el ejemplo de usuarios, se ejecutaría el siguiente código por cáda linea de tweets del CSV:


In [ ]:
def armar_lista_usuario_tweet_id(self, linea, lista_de_pares: list) -> None:
        id_tweet = linea['id']
        usuario = linea['username']
        self._user_id = self.agregar_a_diccionario_terminos(usuario, self._user_id, self._user_to_user_id)
        lista_de_pares.append((self._user_to_user_id[usuario], id_tweet))

Como se ve, se guarda en un diccionario el usuario con su user_id en el diccionario user_to_user_id. Allí la clave será el usuario y el valor será un número de identificación único.
Por otro lado, se agrega a la lista el par (id_usuario, id_tweet). Luego esta lista es intercalada (por el id_usuario) y se agrupan todos los id_tweets de ese usuario en una lista. De esta forma, si tenemos en el diccionario user_to_user_id:

```json
{ "Juan": 0, "Pedro": 1 }
```

Si el posting tiene:

```json
["924"]["134","224","341"]
```

Sabremos que Juan publico el tweet 924 y Pedro los tweets 134, 224 y 341, debido a que las listas se encuentran ordenadas y corresponden su posición al user_id de cada uno. Y lo mismo ocurre con los otros indices.







Pero el gran problema era, _¿Cómo pasamos la lista de pares sin tenerla guardada en memoria?_

Para eso, hicimos que al guardar un bloque intermedio a los 10.000 tweets, se guarde un lista con los valores por cada clave y retorne la ruta del archivo generado y la agregue a otra lista. Por lo tanto, nos quedará una lista con strings que serán la ruta que luego leerá el intercalador.


El equipo se vio ante la necesidad de poder unir los CSV ya que, cada integrante descargó durante semanas de forma individual su propio CSV. Para esto se desarrollo el método "unir_csvs" el cuál une a todos en un solo archivo:

In [ ]:
 def unir_csvs(self, ruta_documentos: str) -> None:
        lista_documentos = [self.__join(ruta_documentos, nombre_doc) \
                        for nombre_doc in os.listdir(ruta_documentos) \
                        if os.path.isfile(self.__join(ruta_documentos, nombre_doc))]

        primer_documento = self.__join(ruta_documentos, "unificado.csv")
        os.rename(lista_documentos.pop(), primer_documento)
        with open(primer_documento, "w", encoding="utf-8") as unificado:
            for documento in lista_documentos:
                with open(documento) as doc:
                    doc.readline()
                    shutil.copyfileobj(doc, unificado)

3) BUSCAR

El opción de buscar no funciona como las anteriores, esta te lleva a un nuevo menú en el cual tendrás que volver a elegir una opción:

In [4]:
  def __desplegar_menu_buscador(self):
        print("|************************************************************************|")
        print("| 1) Buscar por palabra                                                  |")
        print("| 2) Buscar por frase                                                    |")
        print("| 3) Buscar por fecha y hora                                             |")
        print("| 4) Buscar por usuario                                                  |")
        print("| 5) Regresar al menú principal                                          |")
        print("| 6) Cerrar programa                                                     |")
        print("|________________________________________________________________________|")
        print()

Allí se tienen los distintos tipos de búsqueda y dependiendo del cual se elija se llevará a una función del objeto Buscador distinta.
Las búsquedas se realizan todas de manera similar. Se llama a un método del Buscador el cual pasará a buscar los tweetID y los devolverá. Para su funcionamiento es necesario que se le pase el nombre de lo que se busca (Usuarios, tweets, fechas o palabras) y lo que se quiere buscar en cuestión (Por ejemplo, un usuario, una fecha, ) Luego este retorno es utilizado en otro método el cual buscará el texto (lo tweeteado) correspondiente a cada tweetID y así podrá devolver lo que es fue tweeteado.

In [ ]:
def __obtener_lista_tweet_id(self, nombre: str, termino: str) -> list:
        ruta_dict = f"./salida/diccionario_{nombre}.json"
        ruta_posting = f"./salida/posting_{nombre}.json"
        with open (ruta_dict, encoding="utf-8") as diccionario:
            data = dict(json.load(diccionario))
        try:
            term_id = int(data[termino])
            with open (ruta_posting, encoding="utf-8") as post:
                linea = ""
                for _ in range(term_id + 1):
                    linea = post.readline()
        except KeyError:
            print(f"No se encontro {termino}") 

            return eval(linea)

In [ ]:
 def __obtener_tweet(self, tweetid : list) -> dict:
        ruta_dict = f"./salida/diccionario_tweets.json"
        ruta_posting = f"./salida/posting_tweets.json"
        with open (ruta_dict, encoding="utf-8") as diccionario:
            data = json.load(diccionario)
            try:
                pos = []
                aux = {}
                for t in tweetid:
                    for tw, posicion in data.items():
                        if int(tw) == int(t):
                            pos.append((posicion, tw))
                with open (ruta_posting, encoding = "utf-8") as post:
                    lineas = post.readlines()
                    for p, tw  in pos:
                        aux[tw] = lineas[int(p)]
            except KeyError:
                print(f"No se encontró el tweet {tw}")
            else:
                return aux
            

_EXCEPCIONES_

Se decidió realizar un manejo de excepciones personalizadas las cuales fueron creadas en el archivo "excepciones.py" y las mismas son: OperacionCanceladaException, FechaInvalidaException y TerminoNoEncontradoException

_CONCLUSIONES Y APRENDIZAJE_

Podemos concluir que, la clase que conecta a todo el resto es la UI (interfaz_usuario.py). Si tendríamos que describir brevemente cada clase y sus componentes principales podríamos decir que:

-UI: Se encarga de solicitar el Input y distribución de tareas.

-TweetDownloader: Se encarga de la descarga de los tweets a través de la TwitterAPI y de la modificación y guardado de los mismos en disco.

-Indexador: Se compone principalmente de: guardar bloques intermedios, invertir bloques, intercalar bloques y guardar (para el posting) y por otro lado el armado de un diccionario para cada posting.

-Buscador: Se compone de un método que obtiene los tweetsID que corresponden a lo buscado y otro que encuentra el texto de cada tweet a partir de su tweetID.


A pesar de los desafíos que se nos presentaron, hemos observado que no hay mejor resolución que la cooperación entre los integrantes. Cuando uno se encontraba sin poder avanzar en cierta parte del programa, con solo consultarlo otro podía dar una idea a pesar de no ser la parte que le correspondía hacer, pero de esta forma se logra avanzar con rapidez. 
Esto, además, nos llevó a darnos cuenta que no solo realizar la tarea que le tocó a cada uno y listo, sino que tambien fue útil revisar lo que le correspondía a los demás, así podíamos tener un pantallazo general de lo que hacía el programa y como se iban a poder relacionar los distintos métodos.

Finalmente, el diagrama de clases nos sirvó para orientarnos. En un principio, habíamos comenzado a escribir código sin tener el diagrama hecho, lo cual, evidentemente, no nos llevó a ningún lugar ya que no podíamos entablar una relación entre cada clase y cada método, por ejemplo, no sabíamos que parámetros esperaba x método de otra clase, lo que nos imposibilitaba el avance del código o nos hacía tener que modificarlo al darnos cuenta (o que nos avisen) que otro método esperaba otro tipo de entrada como parámetro o que iba a retornar algo distinto que no tuvimos en cuenta.
Al implementar el diagrama estos problemas desaparecieron, cada uno mantenía el diagrama actualizado con los cambios que se aplicaban. De esta manera, todos podíamos saber en todo momento que parámetros y retorno tenía cada uno de los métodos.